### Open ICPSR Search

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [2]:
# Get Search Results
url = "https://www.openicpsr.org/openicpsr/search/aea/studies?start=1&ARCHIVE=aea&rows=5000"
r = requests.get(url)

In [3]:
results_text = r.text.split('searchResults : ')[1]

In [4]:
# Find the starting index of the "docs" array
start_index = results_text.find('"docs":[')
if start_index != -1:
    # Extract the content from the "docs" array
    docs_content = results_text[start_index + len('"docs":['):]

    # Find the index of the closing bracket for "docs" array
    end_index = docs_content.rfind(']')
    if end_index != -1:
        # Extract the "docs" array content
        docs_array = docs_content[:end_index + 1]
    else:
        print('Closing bracket for "docs" not found.')
else:
    print('"docs" array not found.')

In [5]:
docs = "[" + docs_array.split(',"numFound"')[0]

In [6]:
dat = pd.json_normalize(json.loads(docs))

In [7]:
dat['SUMMARY'] = dat['SUMMARY'].astype(str).apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
dat['SUMMARY'] = dat['SUMMARY'].str.replace('[', '').str.replace(']', '').str.strip("'")
dat

,DATEUPDATED,OWNER,SUMMARY,TITLE,AUTHOR,ID
0,2024-01-18T00:00:00Z,openicpsr,We run a field experiment offering graduated m...,Data and Code for: Asset- Based Microfinance f...,"[Bari, Faisal, Malik, Kashif, Meki, Muhammad, ...",195984
1,2024-01-18T00:00:00Z,openicpsr,"Code and data accompanying the article ""Pricin...",Data and Code for: Pricing Power in Advertisin...,"[Gentzkow, Matthew, Shapiro, Jesse, Yang, Fran...",193824
2,2024-01-18T00:00:00Z,openicpsr,This paper studies the interplay between genet...,"Data and Code for: ""The Nurture of Nature and ...","[Houmark, Mikkel, Ronda, Victor, Rosholm, Mich...",192844
3,2024-01-18T00:00:00Z,openicpsr,"In the U.S, after age 65, households face inco...","Data and Code for: ""Old Age Risks, Consumption...","[Blundell, Richard, Borella, Margherita, Comma...",193623
4,2024-01-11T00:00:00Z,openicpsr,Without strong assumptions about how noise man...,Data and Code for: Distinguishing Common Ratio...,"[McGranaghan, Christina, Nielsen, Kirby, O'Don...",194003
...,...,...,...,...,...,...
4660,2019-10-11T00:00:00Z,openicpsr,This paper introduces endogenous and directed ...,Replication data for: The Environment and Dire...,"[Acemoglu, Daron, Aghion, Philippe, Bursztyn, ...",112489
4661,2019-10-11T00:00:00Z,openicpsr,"""We estimate an equilibrium model of decision ...",Replication data for: The Value of Information...,"[Iaryczower, Matias, Shum, Matthew]",112491
4662,2019-09-16T00:00:00Z,openicpsr,"""This study examines male and female course pe...","Data and Code for ""The Gender Gap in Undergrad...","[Ahlstrom, Laura J., Asarta, Carlos J.]",111746
4663,2019-09-08T00:00:00Z,openicpsr,This repository holds Matlab codes to compute ...,Matlab Codes for Optimal Collateralized Contracts,"[Cao, Dan]",111022


In [8]:
dat.to_csv("../data/aea_icpsr_repos.csv")

## Get Metadata for Each

In [ ]:
https://pcms.icpsr.umich.edu/pcms/api/1.0/oai/studies?metadataPrefix=oai_ddi25&verb=GetRecord&identifier=195984&version=V1&page=/openicpsr/project/195984/version/V1/view?flag=follow&path=/openicpsr/195984/fcr:versions/V1&type=project&user=gsood07@gmail.com

In [39]:
from webdriver_manager.chrome import ChromeDriverManager

# Initialize Chrome WebDriver with options
#driver = webdriver.Chrome(ChromeDriverManager().install())

# List of identifiers
identifiers = [195984, 123345]

# Setup output directory
out_dir = 'icpsr_metadata'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

for dataset_id in identifiers:
    try:
        # Construct URL
        url = f"https://pcms.icpsr.umich.edu/pcms/api/1.0/oai/studies?metadataPrefix=oai_ddi25&verb=GetRecord&identifier={dataset_id}"

        # Open the URL in the browser
        driver.get(url)

        # Wait for the "I Agree" button and click it
        agree_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'downloadButton'))
        )
        agree_button.click()

        # Wait for the download to complete
        time.sleep(5)  # Adjust the sleep time based on download speed

        # Find the downloaded file and move it to the output directory
        downloaded_file_path = "icpsr_metadata/"  # Replace with the actual path
        new_file_path = os.path.join(out_dir, f"{dataset_id}.xml")
        os.rename(downloaded_file_path, new_file_path)

    except Exception as e:
        print(f"Failed to retrieve metadata for dataset {dataset_id}. Error: {str(e)}")

# Quit the WebDriver
driver.quit()

Failed to retrieve metadata for dataset 195984. Error: HTTPConnectionPool(host='localhost', port=36857): Max retries exceeded with url: /session/d316ccd824a47ebfd17a0b6112b87bb1/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f1775ef67f0>: Failed to establish a new connection: [Errno 111] Connection refused'))
Failed to retrieve metadata for dataset 123345. Error: HTTPConnectionPool(host='localhost', port=36857): Max retries exceeded with url: /session/d316ccd824a47ebfd17a0b6112b87bb1/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f1775ef6b20>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [40]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Initialize Chrome WebDriver with options
#chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')  # Run in headless mode (without GUI)
#driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

# URL of the page
url = "https://pcms.icpsr.umich.edu/pcms/api/1.0/oai/studies?metadataPrefix=oai_ddi25&verb=GetRecord&identifier=195984&version=V1&page=/openicpsr/project/195984/version/V1/view?flag=follow&path=/openicpsr/195984/fcr:versions/V1&type=project&user=gsood07@gmail.com"

# Open the URL in the browser
driver.get(url)

# Wait for the "I Agree" button and click it
agree_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, 'downloadButton'))
)
agree_button.click()

# Wait for the download to complete
time.sleep(5)  # Adjust the sleep time based on download speed

# Find the downloaded file and move it to the output directory
downloaded_file_path = "/path/to/your/downloads"  # Replace with the actual path
new_file_path = os.path.join("icpsr_metadata", "195984.xml")
os.rename(downloaded_file_path, new_file_path)

# Quit the WebDriver
driver.quit()


MaxRetryError: HTTPConnectionPool(host='localhost', port=36857): Max retries exceeded with url: /session/d316ccd824a47ebfd17a0b6112b87bb1/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f1775ef6a60>: Failed to establish a new connection: [Errno 111] Connection refused'))